In [ ]:
pip install unsloth transformers trl

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.6/66.6 kB 6.7 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of torchvision to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 381.1/381.1 kB 32.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 423.1/423.1 kB 43.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.1/59.1 MB 14.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 506.8/506.8 kB 38.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.7/295.7 kB 29.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 122.9/122.9 MB 10.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 899.7/899.7 MB 1.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 170.5/170.5 MB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 594.3/594.3 MB 1.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.2

In [ ]:
import torch
from unsloth import FastLanguageModel
from datasets import load_dataset
from trl import SFTTrainer
from transformers import TrainingArguments
from unsloth.chat_templates import get_chat_template , standardize_sharegpt

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!


In [ ]:
model , tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/Llama-3.2-3B-Instruct",
    max_seq_length = 2048,
    load_in_4bit=True
)

==((====))==  Unsloth 2026.1.2: Fast Llama patching. Transformers: 4.57.3.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.9.1+cu128. CUDA: 7.5. CUDA Toolkit: 12.8. Triton: 3.5.1
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.33.post2. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/2.35G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/234 [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/454 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.2M [00:00<?, ?B/s]

chat_template.jinja: 0.00B [00:00, ?B/s]

In [16]:
model = FastLanguageModel.get_peft_model(
    model, r=16,
    target_modules=["q_proj", "v_proj" , "k_proj" , "o_proj" , "gate_proj" , "up_proj" , "down_proj"]

)

Unsloth: Already have LoRA adapters! We shall skip this step.


In [ ]:
tokenizer = get_chat_template(tokenizer, chat_template="llama-3.1")

In [ ]:
dataset = load_dataset("mlabonne/FineTome-100k" , split="train")

In [ ]:
datset = standardize_sharegpt(dataset)

Unsloth: Standardizing formats (num_proc=2):   0%|          | 0/100000 [00:00<?, ? examples/s]

In [ ]:
dataset

In [ ]:
dataset[0]

In [ ]:
dataset = dataset.map(
    lambda examples: {
        "text": [
            tokenizer.apply_chat_template(
                [{"role": turn["from"], "content": turn["value"]} for turn in convo],
                tokenize=False
            )
            for convo in examples["conversations"]
        ]
    },
    batched = True
)

Map:   0%|          | 0/100000 [00:00<?, ? examples/s]

In [ ]:
dataset

In [ ]:
dataset[0]

In [ ]:
trainer = SFTTrainer(
    model = model,
    train_dataset = dataset,
    dataset_text_field = "text",
    max_seq_length = 2048,
    tokenizer = tokenizer,
    args=TrainingArguments(
        per_device_train_batch_size=2,
        gradient_accumulation_steps=4,
        warmup_steps=5,
        max_steps = 60,
        learning_rate=2e-4,
        fp16= not torch.cuda.is_bf16_supported(),
        bf16=torch.cuda.is_bf16_supported(),
        logging_steps=1,
        output_dir="outputs",
        report_to="none"
    )
)

In [14]:
trainer.train()

The model is already on multiple devices. Skipping the move to device specified in `args`.
==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 100,000 | Num Epochs = 1 | Total steps = 60
O^O/ \_/ \    Batch size per device = 2 | Gradient accumulation steps = 4
\        /    Data Parallel GPUs = 1 | Total batch size (2 x 4 x 1) = 8
 "-____-"     Trainable parameters = 24,313,856 of 3,237,063,680 (0.75% trained)


Unsloth: Will smartly offload gradients to save VRAM!


Step,Training Loss
1,1.478500
2,1.879700
3,1.467200
4,1.490500
5,1.415400
6,1.471600
7,1.001100
8,1.535100
9,1.262000
10,1.270100


Step,Training Loss
1,1.478500
2,1.879700
3,1.467200
4,1.490500
5,1.415400
6,1.471600
7,1.001100
8,1.535100
9,1.262000
10,1.270100


TrainOutput(global_step=60, training_loss=1.035423035422961, metrics={'train_runtime': 385.8762, 'train_samples_per_second': 1.244, 'train_steps_per_second': 0.155, 'total_flos': 5503643452858368.0, 'train_loss': 1.035423035422961, 'epoch': 0.0048})

In [15]:
model.save_pretrained("fine-tuned-model")

In [19]:
inference_model, inference_tokenizer = FastLanguageModel.from_pretrained(
    model_name = "./fine-tuned-model",
    max_seq_length = 2048,
    load_in_4bit = True
)

==((====))==  Unsloth 2026.1.2: Fast Llama patching. Transformers: 4.57.3.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.9.1+cu128. CUDA: 7.5. CUDA Toolkit: 12.8. Triton: 3.5.1
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.33.post2. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


In [21]:
text_prompts = [
    "What are the key principles of investment?"
]
for prompt in text_prompts:
  formatted_prompt = inference_tokenizer.apply_chat_template([{
      "role": "user",
      "content": prompt
  }], tokenize = False)
  model_inputs = inference_tokenizer(formatted_prompt, return_tensors="pt").to("cuda")
  generated_ids = inference_model.generate(
      **model_inputs,
      max_new_tokens = 512,
      temperature = 0.7,
      do_sample = True,
      pad_token_id = inference_tokenizer.pad_token_id
  )
response = inference_tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]
print(response)

system

Cutting Knowledge Date: December 2023
Today Date: 05 Jan 2026

user

What are the key principles of investment?gpt

The key principles of investment include:

1. Diversification: Investing in a variety of assets helps to minimize risk and maximize potential returns.
2. Risk management: Understanding and managing risk is crucial in investing, as it can help to protect investments and achieve long-term goals.
3. Time horizon: Investing requires a long-term perspective, as it can take years or even decades for investments to mature.
4. Diversification: Investing in a variety of assets helps to minimize risk and maximize potential returns.
5. Time horizon: Investing requires a long-term perspective, as it can take years or even decades for investments to mature.
6. Diversification: Investing in a variety of assets helps to minimize risk and maximize potential returns.
7. Risk management: Understanding and managing risk is crucial in investing, as it can help to protect investments 